In [1]:
import sys,os,copy,glob
import numpy as np
sys.path.append(os.path.expanduser('~/smodels'))
from builder.protomodel import ProtoModel
from builder.manipulator import Manipulator
from tester.predictor import Predictor
from tester.combiner import Combiner
from walker.hiscore import Hiscore
from smodels.experiment.databaseObj import Database
from smodels.tools import runtime
from smodels.tools.physicsUnits import fb
runtime._experimental = True
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

In [2]:
def fromDict(inputDict):
    
    p = ProtoModel(walkerid=0)
    for key,v in inputDict.items():
        setattr(p,key,copy.deepcopy(v))
        
    return p

In [6]:
#Get highest score from the run:
with open('./real3.dict','r') as f:
    pList = eval(f.read())
run = eval(os.path.basename(ff).replace('real','').replace('.dict',''))
pList = [fromDict(pDict) for pDict in pList[:]]
p = sorted(pList, key = lambda p: p.K, reverse=True)[0]
protomodel = p

In [7]:
print(run,protomodel)    

3 ProtoModel (XZ1,Xt,Xc): K = 8.77, Z = 3.51


In [8]:
# pred = Predictor(walkerid=0,dbpath='./smodels-database-noFastlim/')
pred = Predictor(walkerid=0,dbpath='./official123_smodels200.pcl')
res = pred.predict(protomodel)
oldPrior = pred.combiner.computePrior(protomodel,name='expo2')
newPrior = pred.combiner.computePrior(protomodel,name='expo1')

[combine:0] selected predictions down via SRs from 151 to 21.
[combine:0] sorting out subsets, reduced 3338 -> 405 combinations.


combination #405/405 100%|||||||||||||||||||||||||||||||||||||||||Time: 0:00:28


[combine:0] selected predictions down via SRs from 151 to 21.
[combine:0] sorting out subsets, reduced 3338 -> 405 combinations.


combination #405/405 100%|||||||||||||||||||||||||||||||||||||||||Time: 0:00:31


In [9]:
print('new K = ',protomodel.K)

new K =  7.522839222989878


In [13]:
p = protomodel
print('\n run %i: K = %1.2f, Z = %1.2f, muhat = %1.2f, mumax = %1.2f' %(run,p.K,p.Z,p.muhat,p.mumax))
print('\t Best Combo:')
for tp in sorted(p.bestCombo, key = lambda tp: tp.expResult.globalInfo.id):
    if tp.dataset.dataInfo.dataType == 'efficiencyMap':
        print('\t\t',tp.expResult.globalInfo.id,tp.txnames,tp.dataset.dataInfo.dataId,
          tp.dataset.dataInfo.observedN,tp.dataset.dataInfo.expectedBG,tp.dataset.dataInfo.bgError,
              'pred=',tp.xsection.value,'UL =',tp.getUpperLimit())
    else:
        print('\t\t',tp.expResult.globalInfo.id,tp.txnames,tp.dataset.dataInfo.dataId,
              'pred=',tp.xsection.value,'UL =',tp.getUpperLimit())

print('\t Constraints:')
for tp in sorted(p.tpList, key = lambda x: x[0], reverse=True):
    if tp[0] < 1.0: continue
    print('\t\t r = %1.2f' %tp[0],tp[2].expResult.globalInfo.id,tp[2].dataset.dataInfo.dataType,tp[2].txnames,
          'theory xsec =  %1.2f, UL = %1.2f, UL_exp = %1.2f' 
          %(tp[2].xsection.value.asNumber(fb),tp[2].upperLimit.asNumber(fb),tp[2].expectedUL.asNumber(fb)))



 run 3: K = 7.52, Z = 3.32, muhat = 1.00, mumax = 1.00
	 Best Combo:
		 ATLAS-SUSY-2013-02 [T2] SR6jtp 6 4.9 1.6 pred= 9.19E-06 [pb] UL = 3.99E-01 [fb]
		 ATLAS-SUSY-2013-15 [T2tt] tNboost 5 3.3 0.7 pred= 1.52E-05 [pb] UL = 3.73E-01 [fb]
		 ATLAS-SUSY-2015-02 [T2tt] SR1 12 5.5 0.72 pred= 3.94E-04 [pb] UL = 4.40E+00 [fb]
		 ATLAS-SUSY-2016-07 [T2] 2j_Meff_1200 611.0 526.0 31.0 pred= 1.15E-03 [pb] UL = 4.84E+00 [fb]
		 CMS-SUS-13-012 [T2] None pred= 2.13E-03 [pb] UL = 3.80E+01 [fb]
		 CMS-SUS-16-050 [T2tt] None pred= 4.60E-02 [pb] UL = 1.79E+02 [fb]
	 Constraints:
		 r = 1.30 CMS-SUS-19-006 upperLimit [T2tt] theory xsec =  45.99, UL = 35.38, UL_exp = 55.02
		 r = 1.14 CMS-SUS-16-033 upperLimit [T2] theory xsec =  23.85, UL = 20.86, UL_exp = 30.81
		 r = 1.13 CMS-SUS-19-006 upperLimit [T2] theory xsec =  23.85, UL = 21.14, UL_exp = 23.70
